In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import sklearn.metrics

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report


import lightgbm as lgb

from tqdm import tqdm

%matplotlib inline
sns.set(style='darkgrid', font_scale=1.5)

In [2]:
data = pd.read_csv('../datasets/added_pos1_metrics_windowed.csv', index_col=[0, 1])
data = data.droplevel(0)
data.head()

,x_mean,x_std,x_max,x_min,x_energy,x_iqr,y_mean,y_std,y_max,y_min,...,z_iqr_pos1,xy_corr_pos1,xz_corr_pos1,yz_corr_pos1,height,mass,position,is_valid,age,sex
level_1,,,,,,,,,,,,,,,,,,,,,
1673,11156.76,123.532772,11380.0,10836.0,1.244884e+08,177.0,1158.80,73.172275,1348.0,920.0,...,485.0,-0.203383,0.750925,-0.452449,185,75,1,0,18,1
1698,11156.88,128.858961,11380.0,10836.0,1.244924e+08,193.0,1137.96,69.416023,1300.0,920.0,...,485.0,-0.203383,0.750925,-0.452449,185,75,1,0,18,1
1723,11154.36,134.410311,11532.0,10836.0,1.244376e+08,190.0,1088.32,171.829072,1380.0,164.0,...,485.0,-0.203383,0.750925,-0.452449,185,75,1,0,18,1
1748,10998.28,415.506022,11532.0,9384.0,1.211331e+08,323.0,853.36,499.190851,1380.0,-1032.0,...,485.0,-0.203383,0.750925,-0.452449,185,75,1,0,18,1
1773,10883.04,409.575930,11532.0,9384.0,1.186066e+08,543.0,537.68,622.192456,1380.0,-1032.0,...,485.0,-0.203383,0.750925,-0.452449,185,75,1,0,18,1


In [3]:
data = data.drop(['is_valid', 'age', 'sex'], axis=1)

In [4]:
y = data['position']
X = data.drop('position', axis=1)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.7, random_state=1234
)

params_grid = {
            'n_estimators': [100, 200, 300, 500],
            'max_depth' : np.arange(3, 7),
            'learning_rate': np.linspace(0.0125, 0.6, 10)
        }

In [16]:
lgb_estimator = lgb.LGBMClassifier(boosting_type='gbdt', 
                                   objective='multiclass')

gsearch = GridSearchCV(estimator=lgb_estimator, param_grid=params_grid, cv=5, verbose=10, n_jobs=3)
lgb_model = gsearch.fit(X=X_train, y=y_train)

Fitting 5 folds for each of 160 candidates, totalling 800 fits


In [17]:
lgb_model.best_params_

{'learning_rate': 0.4694444444444445, 'max_depth': 4, 'n_estimators': 200}

In [18]:
print('train accuracy {:.4f}'.format(
    accuracy_score(lgb_model.predict(X_train), y_train)
))
print('test accuracy {:.4f}'.format(
    accuracy_score(lgb_model.predict(X_test), y_test)
))

train accuracy 1.0000
test accuracy 0.9989
